In [1]:
import pandas as pd
import os 
from functions import to_minute, do_checkout, unique_customers_and_combine_files,create_next_location, add_entrance
from class_customer import Customer
from datetime import timedelta
import seaborn as sns
import numpy as np

In [2]:
#create df with all days 
df = unique_customers_and_combine_files(path= 'data/')
df.head(), df.shape, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24877 entries, 0 to 24876
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    24877 non-null  datetime64[ns]
 1   customer_no  24877 non-null  int64         
 2   location     24877 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 583.2+ KB


(            timestamp  customer_no location
 0 2019-09-02 07:03:00            1    dairy
 1 2019-09-02 07:03:00            2    dairy
 2 2019-09-02 07:04:00            3    dairy
 3 2019-09-02 07:04:00            4    dairy
 4 2019-09-02 07:04:00            5   spices,
 (24877, 3),
 None)

In [3]:
# add entrance rows to the df
df = add_entrance(df)

In [4]:
#transform df on minute base
df = to_minute(df)
df.head()

,customer_no,timestamp,location
0,1,2019-09-02 07:02:00,entrance
1,1,2019-09-02 07:03:00,dairy
2,1,2019-09-02 07:04:00,dairy
3,1,2019-09-02 07:05:00,checkout
4,2,2019-09-02 07:02:00,entrance


In [5]:
#transform df by adding checkouts
df = do_checkout(df)
df.tail()

,customer_no,timestamp,location
61371,7438,2019-09-06 21:51:00,checkout
61372,7440,2019-09-06 21:51:00,checkout
61373,7441,2019-09-06 21:51:00,checkout
61374,7444,2019-09-06 21:51:00,checkout
61375,7445,2019-09-06 21:51:00,checkout


In [6]:
#transform to add next location
df = create_next_location(df)
df.head()

,customer_no,timestamp,location,next_location
7445,1,2019-09-02 07:02:00,entrance,dairy
0,1,2019-09-02 07:05:00,checkout,checkout
7447,1,2019-09-02 07:04:00,dairy,checkout
7446,1,2019-09-02 07:03:00,dairy,dairy
7448,2,2019-09-02 07:02:00,entrance,dairy


In [15]:
#create crosstab with probabilities
P = pd.crosstab(index=df['location'],columns=df['next_location'],normalize='index').round(3)
P

next_location,checkout,dairy,drinks,entrance,fruit,spices
location,,,,,,
checkout,1.000,0.000,0.000,0.000,0.000,0.000
dairy,0.095,0.599,0.052,0.104,0.085,0.065
drinks,0.139,0.101,0.455,0.115,0.095,0.094
entrance,0.169,0.253,0.153,0.033,0.265,0.127
fruit,0.137,0.115,0.078,0.149,0.466,0.056
spices,0.125,0.190,0.147,0.138,0.108,0.292


In [16]:
#check if sum = 1
states = df['location'].unique()
for state in states:
    print(f'Sum of Prob for {state} is {P.loc[state,:].sum()}')

Sum of Prob for entrance is 1.0
Sum of Prob for checkout is 1.0
Sum of Prob for dairy is 1.0
Sum of Prob for spices is 1.0
Sum of Prob for drinks is 0.999
Sum of Prob for fruit is 1.0010000000000001


In [9]:
# Test Customer Class
c = Customer(123)
print(c.active)
print(c.location)
c.move()
print(c.location)

True
entrance
spices
